In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import numpy as np
import random
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from tqdm import tqdm

#set seeds

def get_models(_config):
    pretrained_weights_path = _config['pretrained_weights_path']
    device = _config['device']
    state_dict = torch.load(pretrained_weights_path)
    out_features, in_features = state_dict['fc.weight'].shape
    model = torchvision.models.resnet50(pretrained=False)
    old_model = torchvision.models.resnet50(pretrained=False)
    model.fc = torch.nn.Linear(in_features, out_features)
    old_model.fc = torch.nn.Linear(in_features, out_features)
    model.load_state_dict(state_dict)
    old_model.load_state_dict(state_dict)
    old_model.eval()
    
    return model, old_model

def get_loss(_config):
    if _config['loss'] == "CrossEntropy":
        loss = nn.CrossEntropyLoss()
    return loss

def seed_all(seed):
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed_all(seed)
	random.seed(seed)
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.enabled = True
 
def kaiming_normal_init(m):
	if isinstance(m, nn.Conv2d):
		nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
	elif isinstance(m, nn.Linear):
		nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')


_config = {
    'data_path': 'horse',
    'num_epochs': 100,
	'T': 2,
	'alpha': 0.01,
	'num_new_class': 1,
	'device': 'cuda' if torch.cuda.is_available() else 'cpu',
	'lr': 1e-2,
	'momentum': 0.9,
	'weight_decay': 5e-4,
	'optimizer': 'SGD',
	'pretrained_weights_path': 'pretrained_weights/net.pth',
	'loss': 'CrossEntropy',
	'batch_size': 4,
	'num_workers': 4,
 
}

/home/alexander/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [2]:
class CustomDataset(Dataset):
    def __init__(self, ds_type='train', data_path=None, imgs_list=None):
        self.ds_type = ds_type
        if ds_type == 'train':
            self.transforms = transforms.Compose([transforms.Resize(size=(224,224)),
                                                  transforms.RandomHorizontalFlip(),
                                                  transforms.ColorJitter(brightness=0.4, saturation=0.4, hue=0.4),
                                                  transforms.RandomRotation(degrees=30), 
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        else:
            self.transforms = transforms.Compose([transforms.Resize(size=(224,224)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        
        self.data_path = data_path # path to dir with images
        self.class_name = self.data_path.split('/')[-1]
        self.images_names = imgs_list
        
        
    def __len__(self):
        return len(self.images_names)
    def __getitem__(self, index):
        img_name = self.images_names[index]
        img_path = os.path.join(self.data_path, img_name)
        label = torch.tensor(0)
        img = Image.open(img_path)
        img = self.transforms(img)
        return img, label
        
        
def get_dataloader(train_imgs_list, test_imgs_list, _config):
    train_dataset = CustomDataset(ds_type='train', data_path=_config['data_path'], imgs_list=train_imgs_list)
    test_dataset =  CustomDataset(ds_type='test', data_path=_config['data_path'], imgs_list=test_imgs_list)
    train_loader = DataLoader(train_dataset, batch_size=_config['batch_size'], num_workers=_config['num_workers'], shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=_config['batch_size'], num_workers=_config['num_workers'], shuffle=False)
    return train_loader, val_loader


# prepare for learning

In [3]:
def prepare_model(_config):
    num_new_class = _config['num_new_class']
    device = _config['device']
    model, old_model = get_models(_config)
    in_features = model.fc.in_features
    out_features = model.fc.out_features
    weight = model.fc.weight.data
    bias = model.fc.bias.data
    new_out_features = out_features + num_new_class
    new_fc = nn.Linear(in_features, new_out_features)
    kaiming_normal_init(new_fc)
    new_fc.weight.data[:out_features] = weight
    new_fc.bias.data[:out_features] = bias
    model.fc = new_fc
    model = model.to(device)
    old_model = old_model.to(device)
    cudnn.benchmark = True
    return model, old_model


def get_optimizer(model, _config):
    if _config['optimizer'] == 'SGD':
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=_config['lr'],
                                    momentum=_config['momentum'], weight_decay=_config['weight_decay'])
    if _config['optimizer'] == 'AdamW':
        optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=_config['lr'], weight_decay=_config['weight_decay'])
    
    return optimizer


def train_one_epoch(epoch, model, old_model, train_loader, optimizer, criterion, _config):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    device = _config['device']
    T = _config['T']
    alpha = _config['alpha']
    out_features  = old_model.fc.out_features
    pbar = tqdm(train_loader)
    for batch_idx, (inputs, targets) in enumerate(pbar):
        inputs, targets = inputs.to(device), targets.to(device)
        targets += out_features
        optimizer.zero_grad()
        outputs = model(inputs)
        soft_target = old_model(inputs)
        loss1 = criterion(outputs, targets)
        
        outputs_S = F.softmax(outputs[:,:out_features]/T,dim=1)
        outputs_T = F.softmax(soft_target[:,:out_features]/T,dim=1)
        
        loss2 = outputs_T.mul(-1*torch.log(outputs_S))
        loss2 = loss2.sum(1)
        loss2 = loss2.mean()*T*T
        loss = loss1*alpha+loss2*(1-alpha)
        loss.backward(retain_graph=True)
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        pbar.set_description(f"Epoch: {epoch} | Loss: {train_loss / (batch_idx+1)} | Acc: {100. * correct /total}")
        
    return train_loss / (batch_idx+1)


def eval_one_epoch(epoch, model, test_dataloader):
    model.eval()
    with torch.no_grad():
        pass
        
        
    
    
    
def create_markup(_config):
    imgs_list = os.listdir(_config['data_path'])
    n = len(imgs_list)
    train_len = int(0.9*n)
    random.shuffle(imgs_list)
    train_imgs_list = imgs_list[:train_len]
    test_imgs_list = imgs_list[train_len:]
    return train_imgs_list, test_imgs_list
    
def run_experiment(_config):
    model, old_model = prepare_model(_config)
    train_imgs_list, test_imgs_list = create_markup(_config)
    train_dataloader, val_dataloader = get_dataloader(train_imgs_list, test_imgs_list, _config)
    optimizer = get_optimizer(model, _config)
    
    loss = get_loss(_config)
    for epoch in range(_config['num_epochs']):
        train_one_epoch(epoch, model, old_model, train_dataloader, optimizer, loss, _config)


In [4]:
run_experiment(_config)

/home/alexander/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/alexander/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
  0%|          | 0/7 [00:15<?, ?it/s]


KeyboardInterrupt: 